In [1]:
# 1) find the target kernel name and
# 2) read gpu trace file
# 3) read gpu metrics
# 4) rearrange the information and combine them in the same dataframe
# 5) save the result as csv file

In [2]:
%load_ext autoreload
%autoreload 2

import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg, add the modules
import operator # sorting
from math import *
import matplotlib.pyplot as plt

sys.path.append('../')

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

import prep

In [3]:
bs_size = [32,64,128,256,512,1024]

target_kern_name = 'vectoradd'

out_columns = None
df_kern = None
count = 0

for bs in bs_size:
    trace_file = 'trace'+str(bs)+'.csv'
    metrics_file = 'metrics'+str(bs)+'.csv'
    
    # read trace and metrics into dataframe, from generated trace file
    df_kernel_trace, df_metrics = prep.Prep_trace_metrics(trace_file, metrics_file)
    
    if count == 0:
        out_columns = prep.init_df_columns(df_metrics)
        # empty df
        df_kern = pd.DataFrame(columns=out_columns)
    
    # current kernel trace
    df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
    
    # add current trace
    df_kern = df_kern.append(df_current, ignore_index=True)


    count += 1


In [4]:
df_kern

,kern_name,duration_ms,gridx,gridy,gridz,blkx,blky,blkz,reg_per_thread,shared_mem,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,vectoradd,0.007232,1563.0,1.0,1.0,32.0,1.0,1.0,8.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),0.823646,0.283296%,0.000000%
1,vectoradd,0.004512,782.0,1.0,1.0,64.0,1.0,1.0,8.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.583276,0.412679%,0.000000%
2,vectoradd,0.004096,391.0,1.0,1.0,128.0,1.0,1.0,8.0,0.0,...,Mid (4),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.998382,0.420733%,0.000000%
3,vectoradd,0.004288,196.0,1.0,1.0,256.0,1.0,1.0,8.0,0.0,...,Mid (4),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),2.079424,0.436911%,0.000000%
4,vectoradd,0.004512,98.0,1.0,1.0,512.0,1.0,1.0,8.0,0.0,...,Mid (4),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),2.001535,0.427193%,0.000000%
5,vectoradd,0.005184,49.0,1.0,1.0,1024.0,1.0,1.0,8.0,0.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.905064,0.394786%,0.000000%


In [5]:
df_kern.to_csv('results.csv', index=False, encoding='utf-8')